In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input

reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

sentiment = np.array([1,1,1,1,1,0,0,0,0,0])



The one_hot function in tensorflow.keras.preprocessing.text does word hashing (not the full one-hot encoding you might expect from NumPy).
   It takes a string of text → "amazing restaurant".
   It splits it into words → ["amazing", "restaurant"].
   Each word is mapped to an integer index using a hashing function.
   The integer will be between 1 and the given vocabulary size (n = 30 in your example).
   The hashing is deterministic: same word → same index every time.
   Different words may collide (map to the same number), since it’s hashing.

In [2]:
one_hot("amazing restaurant",30)

[28, 9]

In [3]:
vocab_size = 30
encoded_reviews = [one_hot(d, vocab_size) for d in reviews] # List Comprehension
encoded_reviews

[[23, 10],
 [28, 9],
 [12, 16],
 [10, 27, 17],
 [11, 17, 24],
 [21, 10],
 [11, 17, 3],
 [23, 12],
 [23, 20],
 [19, 3]]

In [4]:
# All the input texts has uneven size inputs -> So we do PADDING -> to make a maximum fixed size vector -> all embedded words has same vector length
# By this , model works efficiently.
max_length = 4
padded_reviews = pad_sequences(encoded_reviews, maxlen=max_length, padding='post')
print(padded_reviews)

[[23 10  0  0]
 [28  9  0  0]
 [12 16  0  0]
 [10 27 17  0]
 [11 17 24  0]
 [21 10  0  0]
 [11 17  3  0]
 [23 12  0  0]
 [23 20  0  0]
 [19  3  0  0]]


Attributes explained:

1. vocab_size

Size of your vocabulary.

In your case: 30.

Meaning: embedding layer expects word indices from 1 → 30 (Given vocab size).

2. embeded_vector_size (also called output_dim)

The size of the vector for each word.

Example: if embeded_vector_size = 5, then each word index is mapped to a 5-dimensional vector like [0.12, -0.07, 0.55, 0.33, -0.9].

You choose this size (common values: 50, 100, 300 depending on dataset size).

Bigger size → captures more semantic detail, but requires more computation.

3. input_length

Length of each input sequence (number of words in a review).

You set this to max_length = 4.

So every review is exactly 4 words long (after padding/truncating).

🔹 2. Why do we pad sequences?

Real reviews (sentences) have different lengths:

"Amazing food" → 2 words

"Service was great, loved it" → 5 words

But neural networks need fixed-size inputs (like an image always has the same width × height).
Makes all reviews exactly max_length (here = 4).

If shorter → add 0s (padding).

If longer → truncate.

🔹 3. Why give embedding size different from input length?

This is a common confusion. Let’s clarify:

Input length (max_length) = number of words in each review.

Embedding size (embeded_vector_size) = number of features used to represent each word.

👉 Example:

A review: "amazing restaurant service nice" (4 words).

After encoding: [3, 7, 12, 9].

Embedding layer maps each index to a 5D vector (since embeded_vector_size = 5).

🔹 4. Why not make embedding size = input length?

Because they serve different purposes:

Input length = how many words you process in a sentence.

Embedding size = how detailed each word representation is.

In [5]:
embeded_vector_size = 5

# Now Keras is smart enough to infer the sequence input length automatically from your input, so you can safely remove input_length.
model = Sequential()
model.add(Input(shape=(max_length,)))
model.add(Embedding(input_dim=vocab_size, output_dim=embeded_vector_size, name="embedding"))
model.add(Flatten()) # to combiine all the embedded vectors to a single flatten vector, so that we can pass it to ANN network. 
model.add(Dense(1, activation='sigmoid'))

In [6]:
x = padded_reviews
y = sentiment

In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())
model.fit(x, y, epochs=50, verbose=0)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 4, 5)                │             150 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 171 (684.00 B)

 Trainable params: 171 (684.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [8]:
# evaluate the model
loss, accuracy = model.evaluate(x,y)
accuracy

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.9000 - loss: 0.6438


0.8999999761581421

In [9]:
weights = model.get_layer('embedding').get_weights()[0] # extracts the embedding matrix from that list.
weights, len(weights)

(array([[-0.00172095, -0.03296547,  0.03338961, -0.03135638, -0.04979312],
        [-0.04114653,  0.04892534, -0.01560957,  0.00348396, -0.02617691],
        [-0.0377836 , -0.02094356,  0.00520033,  0.02116619,  0.01842507],
        [ 0.0888855 , -0.0740597 , -0.0238796 ,  0.03314437, -0.11607344],
        [-0.01881489, -0.03655418,  0.04949501, -0.00769633, -0.04202037],
        [ 0.00609331, -0.03501867,  0.0439705 , -0.00453931, -0.00630636],
        [-0.02187084, -0.01523805, -0.04102244, -0.04566239, -0.0273971 ],
        [-0.03322246,  0.0428693 ,  0.01778976, -0.01179652, -0.04652549],
        [ 0.02847895,  0.00892083,  0.02931624, -0.01875533,  0.04594996],
        [-0.04767699,  0.0342176 ,  0.0366229 , -0.07463325,  0.01019223],
        [ 0.01329623, -0.00126175, -0.0109999 , -0.0159645 , -0.0084962 ],
        [ 0.00247509,  0.0097402 ,  0.03881373, -0.03589834, -0.0371456 ],
        [ 0.04881279, -0.09162681, -0.11176376, -0.06457805, -0.09377867],
        [ 0.01448018,  0.

In [10]:
weights[13] # embedding vector of one-hot encoded word from the sentence = (13) index

array([0.01448018, 0.04854338, 0.03360989, 0.03643483, 0.01628537],
      dtype=float32)